In [10]:
# Import PadreApp from a package
from padre.app import pypadre
# All interactions with padre should be done via the app

In [11]:
# Configuration of padreapp via a configuration file including backend, http, server authentication etc
"""
 from padre.app.padre_app import pypadre
 assert pypadre.offline
 pypadre.authenticate("test", "mgrani") # authentication sets the pypadre app into online mode
 pypadre.config.save()
 assert pypadre.offline == False
 # delete all datasets
 for ds in pypadre.datasets.list():
     pypadre.datasets.delete(ds)
 assert len([ds for ds in pypadre.datasets.list()])==0 # after deletion there should be no datasets
 # fetch downloads. This should include the datasets that are currently supported.
 downloads = pypadre.datasets.search_downloads("Iris", type="multivariate") # search for external datasets
 # user can manipulate downloads here
 downloaded = pypadre.datasets.download(downloads)
 for download in downloaded: 
    pypadre.datasets.put(ds)
 available = pypadre.datasets.list("Iris")
 assert downloads.equals(available) # the check might be more complex here, but both sets should be equal
 dataset = pypadre.datasets.get(available[0])
"""

'\n from padre.app.padre_app import pypadre\n assert pypadre.offline\n pypadre.authenticate("test", "mgrani") # authentication sets the pypadre app into online mode\n pypadre.config.save()\n assert pypadre.offline == False\n # delete all datasets\n for ds in pypadre.datasets.list():\n     pypadre.datasets.delete(ds)\n assert len([ds for ds in pypadre.datasets.list()])==0 # after deletion there should be no datasets\n # fetch downloads. This should include the datasets that are currently supported.\n downloads = pypadre.datasets.search_downloads("Iris", type="multivariate") # search for external datasets\n # user can manipulate downloads here\n downloaded = pypadre.datasets.download(downloads)\n for download in downloaded: \n    pypadre.datasets.put(ds)\n available = pypadre.datasets.list("Iris")\n assert downloads.equals(available) # the check might be more complex here, but both sets should be equal\n dataset = pypadre.datasets.get(available[0])\n'

In [12]:
# List all available datasets(Toy Datasets)
print(pypadre.datasets.list())

[]


In [13]:
# List all currently available local experiments
print(pypadre.experiments.list_experiments())

['Test Experiment SVM', 'iPython Experiment1 ', 'IPython Experiment 3', 'Executor Test 3', 'Test', 'Torch']


In [14]:
# Create a simple experiment via experiment creator utilizing a dataset displayed. Run it with multiple parameters
pypadre.experiment_creator.clear_experiments()
workflow = pypadre.experiment_creator.create_test_pipeline(['isomap embedding'])
pypadre.experiment_creator.create_experiment(name='iPython Experiment1 ', 
                                           description='Simple iPython notebook experiment',
                                           workflow=workflow, 
                                           dataset_list=['Digits'])
pypadre.experiment_creator.execute_experiments()

# Aimed at new users for running already created experiments or running from JSON files

{'condition': False, 'source': <padre.experimentcreator.ExperimentCreator object at 0x7f2f7522f240>, 'message': 'iPython Experiment1  created successfully!'}
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'isomap embedding'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'Isomap Embedding\n'
                             '\n'
                             '    Non-linear dimensionality reduction through '
                             'Isometric Mapping\n'
                             '\n'
                             '    Read more in the :ref:`User Guide '
                             '<isomap>`.\n'
                             '\n'
                             '    Parameters\n'
                             '    ----------\n'
                             '    n_neighbors : integer\n'
                             '        number of neighbors to consider for each '
                   

In [4]:
# Using decorators to run experiments
from padre.ds_import import load_sklearn_toys
from padre import *
from padre.app import pypadre
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

@Workflow(exp_name="Decorated Experiment",
          description="Test experiment with decorators and SVC")
def create_test_pipeline():
    estimators = [('clf', SVC(probability=True))]
    return Pipeline(estimators)


@Dataset(exp_name="Decorated Experiment")
def get_dataset():
    ds = [i for i in load_sklearn_toys()][2]
    return ds



In [5]:
ex = pypadre.experiments.run(decorated=True)
ex = run("Decorated Experiment")  # run the experiment and report
# Aimed at engineers and scientists for quick prototyping and trying out features

creating the workflow
INFO: 2019-01-31 10:25:01.495 Experiment<id:Decorated Experiment>:	start: phase=experiment
INFO: 2019-01-31 10:25:01.496 	Run<id:15c34a10-253a-11e9-839e-1860248c6d22;name:Decorated Experiment>:	start: phase=run
INFO: 2019-01-31 10:25:01.497 		Split<id:0_15c34a11-253a-11e9-839e-1860248c6d22;name:Decorated Experiment>:	start: phase=split
INFO: 2019-01-31 10:25:01.958 		Split<id:0_15c34a11-253a-11e9-839e-1860248c6d22;name:Decorated Experiment>:	stop: phase=split
INFO: 2019-01-31 10:25:01.970 	Run<id:15c34a10-253a-11e9-839e-1860248c6d22;name:Decorated Experiment>:	stop: phase=run
INFO: 2019-01-31 10:25:01.982 Experiment<id:Decorated Experiment>:	stop: phase=experiment
creating the workflow
INFO: 2019-01-31 10:25:02.066 Experiment<id:Decorated Experiment>:	start: phase=experiment
INFO: 2019-01-31 10:25:02.067 	Run<id:15c34a12-253a-11e9-839e-1860248c6d22;name:Decorated Experiment>:	start: phase=run
INFO: 2019-01-31 10:25:02.067 		Split<id:0_15c34a13-253a-11e9-839e-18602

In [7]:
# Creating and running experiments via pure code (example.py)
def create_test_pipeline():
    from sklearn.pipeline import Pipeline
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    # estimators = [('reduce_dim', PCA()), ('clf', SVC())]
    estimators = [('clf', SVC(probability=True))]
    return Pipeline(estimators)



In [8]:
from padre.ds_import import load_sklearn_toys
from padre.core import Experiment
from padre.base import PadreLogger
from padre.eventhandler import add_logger

logger = PadreLogger()
logger.backend = pypadre.repository
add_logger(logger=logger)
ds = [i for i in load_sklearn_toys()][2]
ex = Experiment(name="Test Experiment SVM",
                    description="Testing Support Vector Machines via SKLearn Pipeline",
                    dataset=ds,
                    workflow=create_test_pipeline(), keep_splits=True, strategy="random")
ex.execute()

INFO: 2019-01-30 12:11:13.909 Experiment<id:Test Experiment SVM>:	start: phase=experiment
INFO: 2019-01-30 12:11:13.910 Experiment<id:Test Experiment SVM>:	start: phase=experiment
INFO: 2019-01-30 12:11:13.910 	Run<id:c19ac8b0-247f-11e9-99c1-1860248c6d22;name:Test Experiment SVM>:	start: phase=run
INFO: 2019-01-30 12:11:13.911 	Run<id:c19ac8b0-247f-11e9-99c1-1860248c6d22;name:Test Experiment SVM>:	start: phase=run
INFO: 2019-01-30 12:11:13.911 		Split<id:0_c19ac8b1-247f-11e9-99c1-1860248c6d22;name:Test Experiment SVM>:	start: phase=split
INFO: 2019-01-30 12:11:13.912 		Split<id:0_c19ac8b1-247f-11e9-99c1-1860248c6d22;name:Test Experiment SVM>:	start: phase=split
INFO: 2019-01-30 12:11:14.407 		Split<id:0_c19ac8b1-247f-11e9-99c1-1860248c6d22;name:Test Experiment SVM>:	stop: phase=split
INFO: 2019-01-30 12:11:14.418 		Split<id:0_c19ac8b1-247f-11e9-99c1-1860248c6d22;name:Test Experiment SVM>:	stop: phase=split
INFO: 2019-01-30 12:11:14.432 	Run<id:c19ac8b0-247f-11e9-99c1-1860248c6d22;name:

In [9]:
# The PyTorch wrapper supports the use of User Defined Callbacks 
from padre.core.wrappers.wrapper_pytorch import WrapperPytorch, CallBack
class TestCallbacks(CallBack):
    def on_compute_loss(self, loss):
        print('Function on compute loss. Loss = {loss}'.format(loss=loss))

    def on_epoch_end(self, obj):
        print('Epoch ended')

    def on_epoch_start(self, obj):
        print('Epoch started')

    def on_iteration_start(self, obj):
        print('Iteration started')

    def on_iteration_end(self, obj):
        print('Iteration ended')

In [10]:
# Running an experiment with a custom wrapper
from sklearn.pipeline import Pipeline

# Define the parameters
params = dict()

import json
with open('./tests/proof_of_concept/Pytorch/classification.json') as json_data:
    params = json.load(json_data)

obj = WrapperPytorch(params=params)
obj.set_callbacks([TestCallbacks()])
estimators = [('clf', obj)]
workflow = Pipeline(estimators)
ds = [i for i in load_sklearn_toys()][4]

ex = Experiment(name="Torch",
                description="Testing Torch via SKLearn Pipeline",
                dataset=ds,
                workflow=workflow)
ex.execute()

End of constructor
INFO: 2019-01-30 12:11:23.598 Experiment<id:Torch>:	start: phase=experiment
INFO: 2019-01-30 12:11:23.599 Experiment<id:Torch>:	start: phase=experiment
INFO: 2019-01-30 12:11:23.600 	Run<id:c76141b6-247f-11e9-99c1-1860248c6d22;name:Torch>:	start: phase=run
INFO: 2019-01-30 12:11:23.601 	Run<id:c76141b6-247f-11e9-99c1-1860248c6d22;name:Torch>:	start: phase=run
INFO: 2019-01-30 12:11:23.603 		Split<id:0_c76141b7-247f-11e9-99c1-1860248c6d22;name:Torch>:	start: phase=split
INFO: 2019-01-30 12:11:23.604 		Split<id:0_c76141b7-247f-11e9-99c1-1860248c6d22;name:Torch>:	start: phase=split
Epoch started
Iteration started
Function on compute loss. Loss = 0.3123060667331271
1 0.3123060667331271
Iteration ended
Iteration started
Function on compute loss. Loss = 0.2017887409242478
2 0.2017887409242478
Iteration ended
Iteration started
Function on compute loss. Loss = 0.3139786568236637
3 0.3139786568236637
Iteration ended
Iteration started
Function on compute loss. Loss = 0.2351456

/home/christofer/PycharmProjects/PyPaDRe/venv/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [14]:
# Compare the metrics between experiments and also between runs(Displaying pandas dataframe)
from padre.experimentexecutor import ExperimentExecutor
# Create an experiment with multiple runs
params_pca = {'num_components': [2, 3, 4, 5, 6]}
params_svr = {'C': [0.5, 1.0, 1.5],
              'poly_degree': [1, 2, 3]}
params_dict = {'SVR': params_svr, 'pca': params_pca}
workflow = pypadre.experiment_creator.create_test_pipeline(['pca', 'SVR'])
params_dict = pypadre.experiment_creator.convert_alternate_estimator_names(params_dict)
pypadre.experiment_creator.clear_experiments('IPython Experiment 3')
pypadre.experiment_creator.create_experiment(name='IPython Experiment 3',
                                             description='Experiment to test the comparison of metrics',
                                             dataset_list=['Boston_House_Prices'],
                                             workflow=workflow,
                                             params=params_dict
                                             )
experiments_executor = ExperimentExecutor(experiments=pypadre.experiment_creator.createExperimentList())
experiments_executor.execute(local_run=True, threads=1)
from padre.metrics import CompareMetrics
metrics = CompareMetrics(experiments_list=experiments_executor.experiments)
print(metrics.show_metrics())

{'condition': False, 'source': <padre.experimentcreator.ExperimentCreator object at 0x7f2b98d2e160>, 'message': 'IPython Experiment 3 created successfully!'}
Sequential Execution
Executing experiment: iPython Experiment1 
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'isomap embedding'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'Isomap Embedding\n'
                             '\n'
                             '    Non-linear dimensionality reduction through '
                             'Isometric Mapping\n'
                             '\n'
                             '    Read more in the :ref:`User Guide '
                             '<isomap>`.\n'
                             '\n'
                             '    Parameters\n'
                             '    ----------\n'
                             '    n_neighbors : integer\n'
                             '      

INFO: 2019-01-30 12:15:14.058 		Split<id:0_50529b15-2480-11e9-99c1-1860248c6d22;name:iPython Experiment1 >:	stop: phase=split
INFO: 2019-01-30 12:15:14.058 		Split<id:0_50529b15-2480-11e9-99c1-1860248c6d22;name:iPython Experiment1 >:	stop: phase=split
INFO: 2019-01-30 12:15:14.058 	Run<id:50529b14-2480-11e9-99c1-1860248c6d22;name:iPython Experiment1 >:	stop: phase=run
INFO: 2019-01-30 12:15:14.058 	Run<id:50529b14-2480-11e9-99c1-1860248c6d22;name:iPython Experiment1 >:	stop: phase=run
INFO: 2019-01-30 12:15:14.058 Experiment<id:iPython Experiment1 >:	stop: phase=experiment
INFO: 2019-01-30 12:15:14.058 Experiment<id:iPython Experiment1 >:	stop: phase=experiment
Completed experiment: iPython Experiment1  with execution time: 0.7851390838623047
Executing experiment: IPython Experiment 3
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'principal component analysis'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
  

INFO: 2019-01-30 12:15:14.260 	Run<id:50529b1a-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-01-30 12:15:14.261 	Run<id:50529b1a-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-01-30 12:15:14.262 		Split<id:0_50529b1b-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-01-30 12:15:14.262 		Split<id:0_50529b1b-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-01-30 12:15:14.275 		Split<id:0_50529b1b-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-01-30 12:15:14.275 		Split<id:0_50529b1b-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-01-30 12:15:14.276 	Run<id:50529b1a-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-01-30 12:15:14.276 	Run<id:50529b1a-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-01-30

INFO: 2019-01-30 12:15:14.461 		Split<id:0_50529b33-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-01-30 12:15:14.462 		Split<id:0_50529b33-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-01-30 12:15:14.475 		Split<id:0_50529b33-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-01-30 12:15:14.476 		Split<id:0_50529b33-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-01-30 12:15:14.476 	Run<id:50529b32-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-01-30 12:15:14.476 	Run<id:50529b32-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-01-30 12:15:14.478 	Run<id:50529b34-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-01-30 12:15:14.479 	Run<id:50529b34-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-01-30

INFO: 2019-01-30 12:15:14.675 		Split<id:0_50529b4b-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-01-30 12:15:14.675 		Split<id:0_50529b4b-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-01-30 12:15:14.676 	Run<id:50529b4a-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-01-30 12:15:14.676 	Run<id:50529b4a-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-01-30 12:15:14.677 	Run<id:50529b4c-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-01-30 12:15:14.678 	Run<id:50529b4c-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-01-30 12:15:14.678 		Split<id:0_50529b4d-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-01-30 12:15:14.679 		Split<id:0_50529b4d-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-01-30

INFO: 2019-01-30 12:15:14.881 		Split<id:0_50529b63-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-01-30 12:15:14.881 		Split<id:0_50529b63-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-01-30 12:15:14.882 	Run<id:50529b62-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-01-30 12:15:14.882 	Run<id:50529b62-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-01-30 12:15:14.884 	Run<id:50529b64-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-01-30 12:15:14.885 	Run<id:50529b64-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-01-30 12:15:14.885 		Split<id:0_50529b65-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-01-30 12:15:14.886 		Split<id:0_50529b65-2480-11e9-99c1-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-01-30

In [27]:
# Go online, register new user
# For registering, user has to use the browser to enter details

In [20]:
# Log in to server
# Implemented

In [21]:
# Upload/Sync experiments on demand
# Not Implemented. Experiments can only be uploaded during the execution of the experiment

In [22]:
# Look at the experiment via SPA and check the results and visualize them(if possible)

In [23]:
# Download an experiment from the server and run it.(via the experiment.JSON file) Example for reproducibility
# Not yet implemented

In [ ]:
# Keep track of random seeds used in the experiment

In [ ]:
# Download a dataset from OpenML and upload it to the server
# Generate metrics for uploaded  datasets
# Currently in progress. Some datasets in OpenML do not work and raise exceptions

In [ ]:
# Download a dataset from server
# Implemented

In [ ]:
# Running an experiment with a dataset downloaded from the server and sync the experiment automatically
# Not implemented

In [ ]:
# Running an experiment on the server cluster

In [ ]:
# Tracking the progress of experiments including hyperparameter search